<a href="https://colab.research.google.com/github/railman-misaka/twikit/blob/main/%E3%80%90%E6%94%B9%E4%BF%AE_ReBest%E6%A7%98%E3%80%91241231twikit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ

①セットアップ

In [ ]:
# 必要なライブラリのインストール
!pip install twikit pandas openpyxl

# 必要なインポート
import os
from google.colab import drive
from google.colab import files
import json
import pandas as pd
from datetime import datetime
from twikit import Client
import asyncio
from collections import Counter
from google.colab import output

# Google Driveのマウント
drive.mount('/content/drive')

# ディレクトリ設定
WORK_DIR = '/content/drive/MyDrive/Twitter_Analysis'
COOKIE_DIR = f"{WORK_DIR}/twitter_json"
RESULTS_DIR = f"{WORK_DIR}/profile_results"

# ディレクトリ作成
os.makedirs(COOKIE_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

print("セットアップ完了！")
print(f"Cookie path: {COOKIE_DIR}/cookie_edit.json")
print(f"Results directory: {RESULTS_DIR}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 6.9 MB/s eta 0:00:00
Mounted at /content/drive
セットアップ完了！
Cookie path: /content/drive/MyDrive/Twitter_Analysis/twitter_json/cookie_edit.json
Results directory: /content/drive/MyDrive/Twitter_Analysis/profile_results


②# クッキーファイルのアップロード X連携

# リプライサーチ

In [ ]:
# クッキーファイルのアップロード
def upload_cookie_file():
    print("cookie_edit.jsonファイルをアップロードしてください")
    uploaded = files.upload()

    if 'cookie_edit.json' in uploaded:
        with open(f"{COOKIE_DIR}/cookie_edit.json", 'wb') as f:
            f.write(uploaded['cookie_edit.json'])
        print("クッキーファイルを保存しました")
        return True
    else:
        print("cookie_edit.jsonファイルがアップロードされませんでした")
        return False

upload_cookie_file()

cookie_edit.jsonファイルをアップロードしてください


Saving cookie_edit.json to cookie_edit.json
クッキーファイルを保存しました


True

③初期実行　リプライサーチ

In [ ]:

# TwitterProfileAnalyzerクラスの定義
class TwitterProfileAnalyzer:
    def __init__(self):
        self.client = Client(language='en-US')
        self.cookie_path = f"{COOKIE_DIR}/cookie_edit.json"
        self.results_dir = RESULTS_DIR

    async def setup(self):
        try:
            with open(self.cookie_path, 'r', encoding='utf-8') as file:
                cookies = json.load(file)
            self.client.set_cookies(cookies)
            print("認証に成功しました！")
            return True
        except Exception as e:
            print(f"認証エラー: {e}")
            return False

    async def analyze_user_replies(self, screen_name, tweets_to_analyze=200):
        """指定したユーザーのツイートから、リプライを分析する"""
        try:
            # ユーザー情報を取得
            target_user = await self.client.get_user_by_screen_name(screen_name)
            print(f"{screen_name}のツイートを分析中...")

            # リプライしているユーザーをスクリーンネームベースで追跡
            reply_counter = Counter()
            reply_users = {}

            # ツイートを取得（リプライを含む）
            print("ツイートを取得中...")
            results = await self.client.get_user_tweets(
                target_user.id,
                tweet_type='Replies',  # Repliesタイプに変更
                count=min(tweets_to_analyze, 100)
            )

            analyzed_count = 0
            while results and analyzed_count < tweets_to_analyze:
                for tweet in results:
                    try:
                        # リプライ先のツイートテキストを解析
                        if hasattr(tweet, 'text') and tweet.text.startswith('@'):
                            # @ユーザー名を抽出
                            mentioned_users = [
                                word[1:] for word in tweet.text.split()
                                if word.startswith('@')
                            ]

                            for reply_to in mentioned_users:
                                if reply_to and reply_to != screen_name:
                                    try:
                                        if reply_to not in reply_users:
                                            # スクリーンネームからユーザー情報を取得
                                            try:
                                                reply_user = await self.client.get_user_by_screen_name(reply_to)
                                                reply_users[reply_to] = reply_user
                                                print(f"ユーザー {reply_to} の情報を取得しました")
                                            except Exception as e:
                                                continue
                                        reply_counter[reply_to] += 1
                                    except Exception as e:
                                        print(f"ユーザー {reply_to} の情報取得をスキップ: {e}")
                                        continue
                    except Exception as e:
                        continue

                    analyzed_count += 1
                    if analyzed_count % 20 == 0:
                        print(f"{analyzed_count}件のツイートを分析済み")

                if analyzed_count < tweets_to_analyze and results.next_cursor:
                    try:
                        results = await results.next()
                    except Exception as e:
                        print(f"追加ツイート取得エラー: {e}")
                        break
                else:
                    break

            print(f"\n分析完了: {analyzed_count}件のツイートを処理")
            print(f"リプライ先ユーザー数: {len(reply_users)}人")

            return reply_counter, reply_users

        except Exception as e:
            print(f"分析エラー: {e}")
            return Counter(), {}

    async def get_user_profile(self, user):
        """ユーザーのプロフィール情報を取得する"""
        try:
            # プロフィール情報を辞書形式で整理
            profile_data = {
                'user_id': user.id,                      # ユーザーID
                'name': user.name,                       # 表示名
                'screen_name': user.screen_name,         # @ユーザー名
                'description': user.description,         # プロフィール文
                'location': user.location,               # 場所
                'followers_count': user.followers_count, # フォロワー数
                'following_count': user.following_count, # フォロー数
                'tweets_count': user.statuses_count,    # ツイート数
                'created_at': user.created_at,          # アカウント作成日
                'profile_image_url': user.profile_image_url  # プロフィール画像URL
            }
            return profile_data
        except Exception as e:
            print(f"プロフィール取得エラー: {e}")
            return None

    async def get_user_tweets(self, user, count=1):
        """ユーザーの投稿を取得する"""
        try:
            tweets = []
            results = await user.get_tweets(tweet_type='Tweets', count=count)

            for tweet in results:
                tweet_data = {
                    'tweet_id': tweet.id,                # ツイートID
                    'text': tweet.text,                  # ツイート本文
                    'created_at': tweet.created_at,      # 投稿日時
                    'retweet_count': tweet.retweet_count, # リツイート数
                    'like_count': tweet.favorite_count,   # いいね数
                    'reply_count': tweet.reply_count,     # 返信数
                    'is_retweet': bool(tweet.retweeted_tweet), # リツイートかどうか
                    'is_quote': tweet.is_quote_status     # 引用ツイートかどうか
                }
                tweets.append(tweet_data)

            return tweets
        except Exception as e:
            print(f"ツイート取得エラー: {e}")
            return []

    def save_results(self, frequent_repliers_data, target_screen_name):
        """分析結果をJSONファイルとして保存する"""
        try:
            current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = os.path.join(
                self.results_dir,
                f"analysis_{target_screen_name}_{current_time}.json"
            )

            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(frequent_repliers_data, f, ensure_ascii=False, indent=2)
            print(f"分析結果を保存しました: {filename}")
            return filename
        except Exception as e:
            print(f"保存エラー: {e}")
            return None

    def save_to_excel(self, frequent_repliers_data, target_screen_name):
        """分析結果をExcelファイルとして保存"""
        try:
            # データフレーム用のリストを作成
            rows = []
            for user_data in frequent_repliers_data:
                profile = user_data['profile']
                tweets = user_data['recent_tweets']

                # 最新のツイート3件を結合
                recent_tweets_text = "\n".join(
                    [tweet['text'] for tweet in tweets[:3]]
                ) if tweets else ""

                # 1行のデータとして整形
                row = {
                    '分析対象ユーザー': target_screen_name,
                    '分析日時': datetime.now().strftime("%Y-%m-%d %H:%M"),
                    'ユーザー名': profile['name'],
                    'ユーザーID': f"@{profile['screen_name']}",
                    'プロフィール文': profile['description'],
                    'アカウント作成日': profile['created_at'],
                    'リプライ数': user_data['reply_count'],
                    'フォロワー数': profile['followers_count'],
                    'フォロー数': profile['following_count'],
                    'ツイート数': profile['tweets_count'],
                    '場所': profile['location'],
                    'プロフィール画像URL': profile['profile_image_url'],
                    'アカウントURL': f"https://twitter.com/{profile['screen_name']}",
                    '最近のツイート': recent_tweets_text
                }
                rows.append(row)

            # DataFrameを作成
            df = pd.DataFrame(rows)

             # profile_resultsディレクトリを作成（存在しない場合）
            profile_results_dir = "profile_results"
            os.makedirs(profile_results_dir, exist_ok=True)

            # Excelファイル名を生成
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            excel_file = f"{self.results_dir}/リプライ分析_{target_screen_name}_{timestamp}.xlsx"

            # Excelファイルとして保存
            with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
                df.to_excel(writer, sheet_name='リプライ分析結果', index=False)

                # 列幅の自動調整
                worksheet = writer.sheets['リプライ分析結果']
                for idx, col in enumerate(df.columns):
                    max_length = max(
                        df[col].astype(str).apply(len).max(),
                        len(str(col))
                    )
                    # 最大幅を50文字に制限
                    adjusted_width = min(max_length + 2, 50)
                    worksheet.column_dimensions[chr(65 + idx)].width = adjusted_width

                # 行の高さを調整（最近のツイート用）
                for row in range(2, len(df) + 2):  # Excelは1始まりでヘッダーがあるため+2
                    worksheet.row_dimensions[row].height = 60

            print(f"\nExcelファイルを保存しました: {excel_file}")
            return excel_file

        except Exception as e:
            print(f"Excelファイルの保存中にエラーが発生しました: {e}")
            return None


    async def run_analysis():
        analyzer = TwitterProfileAnalyzer()

        if not await analyzer.setup():
            return

        # ユーザー入力を対話的に取得
        target_user = input("分析対象のユーザー名を入力してください（@を除く）: ")
        min_replies = int(input("最小リプライ数を入力してください（例: 3）: "))
        tweets_to_analyze = int(input("分析するツイート数を入力してください（例: 200）: "))

        print(f"\n{target_user}のリプライを分析します...")
        print(f"- 分析対象ツイート数: {tweets_to_analyze}")
        print(f"- 最小リプライ数: {min_replies}")

        # リプライを分析
        reply_counter, reply_users = await analyzer.analyze_user_replies(target_user, tweets_to_analyze)

        if not reply_counter:
            print("\nリプライが見つかりませんでした。")
            return

        # 頻繁にリプライしているユーザーの情報を収集
        frequent_repliers_data = []
        print("\nリプライの多いユーザーの情報を収集中...")

        for screen_name, reply_count in reply_counter.most_common():
            if reply_count >= min_replies and screen_name in reply_users:
                try:
                    user = reply_users[screen_name]
                    profile_data = await analyzer.get_user_profile(user)
                    tweets = await analyzer.get_user_tweets(user, count=3)

                    if profile_data:
                        user_data = {
                            'profile': profile_data,
                            'reply_count': reply_count,
                            'recent_tweets': tweets
                        }
                        frequent_repliers_data.append(user_data)
                        print(f"@{screen_name}の情報を取得しました（リプライ数: {reply_count}）")
                except Exception as e:
                    print(f"@{screen_name}の情報取得に失敗: {e}")
                    continue

        # 結果を保存
        if frequent_repliers_data:
            analyzer.save_results(frequent_repliers_data, target_user)
            analyzer.save_to_excel(frequent_repliers_data, target_user)
            print("\n分析が完了しました！")
        else:
            print("\n分析対象となるユーザーが見つかりませんでした。")

# Google Colab用の非同期

# Google Colab用の実行コード
output.enable_custom_widget_manager()

# 分析の実行
await run_analysis()

認証に成功しました！
分析対象のユーザー名を入力してください（@を除く）: sora19ai
最小リプライ数を入力してください（例: 3）: 3
分析するツイート数を入力してください（例: 200）: 100

sora19aiのリプライを分析します...
- 分析対象ツイート数: 100
- 最小リプライ数: 3
sora19aiのツイートを分析中...
ツイートを取得中...
ユーザー 1108k4xi の情報を取得しました
20件のツイートを分析済み
ユーザー AquaCleanWater の情報を取得しました
ユーザー kawa11japan の情報を取得しました
40件のツイートを分析済み
60件のツイートを分析済み
80件のツイートを分析済み
ユーザー solt152152 の情報を取得しました
100件のツイートを分析済み

分析完了: 111件のツイートを処理
リプライ先ユーザー数: 4人

リプライの多いユーザーの情報を収集中...

分析対象となるユーザーが見つかりませんでした。


④2回目以降　実行用　リプライサーチ

In [ ]:
async def run_analysis():
        analyzer = TwitterProfileAnalyzer()

        if not await analyzer.setup():
            return

        # ユーザー入力を対話的に取得
        target_user = input("分析対象のユーザー名を入力してください（@を除く）: ")
        min_replies = int(input("最小リプライ数を入力してください（例: 3）: "))
        tweets_to_analyze = int(input("分析するツイート数を入力してください（例: 200）: "))

        print(f"\n{target_user}のリプライを分析します...")
        print(f"- 分析対象ツイート数: {tweets_to_analyze}")
        print(f"- 最小リプライ数: {min_replies}")

        # リプライを分析
        reply_counter, reply_users = await analyzer.analyze_user_replies(target_user, tweets_to_analyze)

        if not reply_counter:
            print("\nリプライが見つかりませんでした。")
            return

        # 頻繁にリプライしているユーザーの情報を収集
        frequent_repliers_data = []
        print("\nリプライの多いユーザーの情報を収集中...")

        for screen_name, reply_count in reply_counter.most_common():
            if reply_count >= min_replies and screen_name in reply_users:
                try:
                    user = reply_users[screen_name]
                    profile_data = await analyzer.get_user_profile(user)
                    tweets = await analyzer.get_user_tweets(user, count=3)

                    if profile_data:
                        user_data = {
                            'profile': profile_data,
                            'reply_count': reply_count,
                            'recent_tweets': tweets
                        }
                        frequent_repliers_data.append(user_data)
                        print(f"@{screen_name}の情報を取得しました（リプライ数: {reply_count}）")
                except Exception as e:
                    print(f"@{screen_name}の情報取得に失敗: {e}")
                    continue

        # 結果を保存
        if frequent_repliers_data:
            analyzer.save_results(frequent_repliers_data, target_user)
            analyzer.save_to_excel(frequent_repliers_data, target_user)
            print("\n分析が完了しました！")
        else:
            print("\n分析対象となるユーザーが見つかりませんでした。")

# Google Colab用の非同期

# Google Colab用の実行コード
output.enable_custom_widget_manager()

# 分析の実行
await run_analysis()

認証に成功しました！
分析対象のユーザー名を入力してください（@を除く）: sora19ai
最小リプライ数を入力してください（例: 3）: 2
分析するツイート数を入力してください（例: 200）: 200

sora19aiのリプライを分析します...
- 分析対象ツイート数: 200
- 最小リプライ数: 2
sora19aiのツイートを分析中...
ツイートを取得中...
ユーザー 1108k4xi の情報を取得しました
20件のツイートを分析済み
ユーザー AquaCleanWater の情報を取得しました
ユーザー kawa11japan の情報を取得しました
40件のツイートを分析済み
60件のツイートを分析済み
80件のツイートを分析済み
ユーザー solt152152 の情報を取得しました
100件のツイートを分析済み
120件のツイートを分析済み
140件のツイートを分析済み
160件のツイートを分析済み
180件のツイートを分析済み
200件のツイートを分析済み

分析完了: 209件のツイートを処理
リプライ先ユーザー数: 4人

リプライの多いユーザーの情報を収集中...
@1108k4xiの情報を取得しました（リプライ数: 2）
分析結果を保存しました: /content/drive/MyDrive/Twitter_Analysis/profile_results/analysis_sora19ai_20241229_220112.json

Excelファイルを保存しました: /content/drive/MyDrive/Twitter_Analysis/profile_results/リプライ分析_sora19ai_20241229_220112.xlsx

分析が完了しました！


In [ ]:
# 非同期で分析を実行する関数
async def run_analysis():
        # TwitterProfileAnalyzerクラスのインスタンスを作成
        analyzer = TwitterProfileAnalyzer()

        # セットアップを実行し、認証が成功したか確認
        if not await analyzer.setup():
            return  # 認証に失敗した場合は関数を終了

        # ユーザー入力を対話的に取得
        target_user = input("分析対象のユーザー名を入力してください（@を除く）: ")  # 分析対象のTwitterユーザー名を入力
        min_replies = int(input("最小リプライ数を入力してください（例: 3）: "))  # リプライの最小数を入力
        tweets_to_analyze = int(input("分析するツイート数を入力してください（例: 200）: "))  # 分析するツイートの数を入力

        # 分析の開始をユーザーに通知
        print(f"\n{target_user}のリプライを分析します...")
        print(f"- 分析対象ツイート数: {tweets_to_analyze}")
        print(f"- 最小リプライ数: {min_replies}")

        # 指定したユーザーのリプライを分析
        reply_counter, reply_users = await analyzer.analyze_user_replies(target_user, tweets_to_analyze)

        # リプライが見つからなかった場合の処理
        if not reply_counter:
            print("\nリプライが見つかりませんでした。")
            return  # 関数を終了

        # 頻繁にリプライしているユーザーの情報を収集するためのリストを初期化
        frequent_repliers_data = []
        print("\nリプライの多いユーザーの情報を収集中...")

        # リプライカウントが多い順にユーザーを処理
        for screen_name, reply_count in reply_counter.most_common():
            # 最小リプライ数以上かつユーザー情報が存在する場合に処理
            if reply_count >= min_replies and screen_name in reply_users:
                try:
                    # リプライ先のユーザーオブジェクトを取得
                    user = reply_users[screen_name]
                    # ユーザーのプロフィール情報を取得
                    profile_data = await analyzer.get_user_profile(user)
                    # ユーザーの最新3件のツイートを取得
                    tweets = await analyzer.get_user_tweets(user, count=3)

                    # プロフィール情報が取得できた場合にデータを構築
                    if profile_data:
                        user_data = {
                            'profile': profile_data,        # ユーザープロフィール情報
                            'reply_count': reply_count,     # リプライ数
                            'recent_tweets': tweets         # 最近のツイート
                        }
                        # 頻繁なリプライユーザーのリストに追加
                        frequent_repliers_data.append(user_data)
                        # 情報取得完了を通知
                        print(f"@{screen_name}の情報を取得しました（リプライ数: {reply_count}）")
                except Exception as e:
                    # ユーザー情報の取得に失敗した場合のエラーメッセージ
                    print(f"@{screen_name}の情報取得に失敗: {e}")
                    continue  # 次のユーザーに進む

        # 収集したデータが存在する場合に結果を保存
        if frequent_repliers_data:
            # JSONファイルとして保存
            analyzer.save_results(frequent_repliers_data, target_user)
            # Excelファイルとして保存
            analyzer.save_to_excel(frequent_repliers_data, target_user)
            # 分析完了を通知
            print("\n分析が完了しました！")
        else:
            # 分析対象のユーザーが見つからなかった場合のメッセージ
            print("\n分析対象となるユーザーが見つかりませんでした。")

# Google Colab用の非同期

# Google Colab用の実行コード
output.enable_custom_widget_manager()  # Google Colabでカスタムウィジェットを有効にする

# 分析の実行
await run_analysis()  # run_analysis関数を非同期で実行


認証に成功しました！
分析対象のユーザー名を入力してください（@を除く）: sora19ai
最小リプライ数を入力してください（例: 3）: 2
分析するツイート数を入力してください（例: 200）: 200

sora19aiのリプライを分析します...
- 分析対象ツイート数: 200
- 最小リプライ数: 2
sora19aiのツイートを分析中...
ツイートを取得中...
ユーザー 1108k4xi の情報を取得しました
20件のツイートを分析済み
ユーザー AquaCleanWater の情報を取得しました
ユーザー kawa11japan の情報を取得しました
40件のツイートを分析済み
60件のツイートを分析済み
80件のツイートを分析済み
ユーザー solt152152 の情報を取得しました
100件のツイートを分析済み


CancelledError: 

⑤2回目以降　実行用　リプライサーチ（X自動化のリプライ検索機能（相手から返信3回以上判定））

In [ ]:
# 非同期で分析を実行する関数
async def run_analysis():
        # TwitterProfileAnalyzerクラスのインスタンスを作成
        analyzer = TwitterProfileAnalyzer()

        # セットアップを実行し、認証が成功したか確認
        if not await analyzer.setup():
            return  # 認証に失敗した場合は関数を終了

        # ユーザー入力を対話的に取得
        target_user = input("分析対象のユーザー名を入力してください（@を除く）: ")  # 分析対象のTwitterユーザー名を入力
        min_replies = int(input("最小リプライ数を入力してください（例: 3）: "))  # リプライの最小数を入力
        tweets_to_analyze = int(input("分析するツイート数を入力してください（例: 200）: "))  # 分析するツイートの数を入力

        # 分析の開始をユーザーに通知
        print(f"\n{target_user}のリプライを分析します...")
        print(f"- 分析対象ツイート数: {tweets_to_analyze}")
        print(f"- 最小リプライ数: {min_replies}")

        # 指定したユーザーのリプライを分析
        reply_counter, reply_users = await analyzer.analyze_user_replies(target_user, tweets_to_analyze)

        # リプライが見つからなかった場合の処理
        if not reply_counter:
            print("\nリプライが見つかりませんでした。")
            return  # 関数を終了

        # 頻繁にリプライしているユーザーの情報を収集するためのリストを初期化
        frequent_repliers_data = []
        print("\nリプライの多いユーザーの情報を収集中...")

        # リプライカウントが多い順にユーザーを処理
        for screen_name, reply_count in reply_counter.most_common():
            # 最小リプライ数以上かつユーザー情報が存在する場合に処理
            if reply_count >= min_replies and screen_name in reply_users:
                try:
                    # リプライ先のユーザーオブジェクトを取得
                    user = reply_users[screen_name]
                    # ユーザーのプロフィール情報を取得
                    profile_data = await analyzer.get_user_profile(user)
                    # ユーザーの最新3件のツイートを取得
                    tweets = await analyzer.get_user_tweets(user, count=3)

                    # 相手からのリプライを取得（Twikitを使用）
                    received_replies = []
                    try:
                        # 検索クエリを構成
                        query = f"from:{screen_name} to:{target_user}"
                        # Twikitのsearch_tweet関数を使用
                        search_results = await analyzer.client.search_tweet(query, "Latest", count=20)

                        for tweet in search_results:
                            # ツイートがリプライであることを確認
                            tweet_data = tweet.__dict__.get('data', {})
                            referenced_tweets = tweet_data.get('referenced_tweets', [])

                            is_reply = any(ref.get('type') == 'replied_to' for ref in referenced_tweets)
                            if is_reply:
                                received_replies.append({
                                    'text': tweet.text,
                                    'created_at': tweet.created_at,
                                    'id': tweet.id
                                })

                            if len(received_replies) >= 3:  # 最大3件まで取得
                                break

                    except Exception as e:
                        print(f"リプライの取得中にエラーが発生: {e}")
                        received_replies = []

                    # プロフィール情報が取得できた場合にデータを構築
                    if profile_data:
                        user_data = {
                            'profile': profile_data,            # ユーザープロフィール情報
                            'reply_count': reply_count,         # リプライ数
                            'recent_tweets': tweets,            # 最近のツイート
                            'received_replies': received_replies  # 相手からのリプライ（新機能）
                        }
                        # 頻繁なリプライユーザーのリストに追加
                        frequent_repliers_data.append(user_data)
                        # 情報取得完了を通知
                        print(f"@{screen_name}の情報を取得しました（リプライ数: {reply_count}）")
                except Exception as e:
                    # ユーザー情報の取得に失敗した場合のエラーメッセージ
                    print(f"@{screen_name}の情報取得に失敗: {e}")
                    continue  # 次のユーザーに進む

        # 収集したデータが存在する場合に結果を保存
        if frequent_repliers_data:
            # JSONファイルとして保存
            analyzer.save_results(frequent_repliers_data, target_user)
            # Excelファイルとして保存
            analyzer.save_to_excel(frequent_repliers_data, target_user)
            # 分析完了を通知
            print("\n分析が完了しました！")
        else:
            # 分析対象のユーザーが見つからなかった場合のメッセージ
            print("\n分析対象となるユーザーが見つかりませんでした。")

# Excelファイルに保存する関数
def save_to_excel(self, data, target_user):
    """分析結果をExcelファイルに保存する"""
    workbook = Workbook()
    sheet = workbook.active
    sheet.title = "分析結果"

    # ヘッダー行を作成
    headers = [
        'ユーザー名', 'スクリーンネーム', 'フォロワー数', 'フォロー数',
        'ツイート数', 'リプライ数', '説明', '場所', 'URL',
        '最新ツイート1', '最新ツイート2', '最新ツイート3',
        '相手からのリプライ1', '相手からのリプライ日時1',
        '相手からのリプライ2', '相手からのリプライ日時2',
        '相手からのリプライ3', '相手からのリプライ日時3'
    ]
    sheet.append(headers)

    # データを行として追加
    for user_data in data:
        profile = user_data['profile']
        recent_tweets = user_data['recent_tweets']
        received_replies = user_data.get('received_replies', [])

        # ツイートとリプライの配列を3件に調整
        tweets = (recent_tweets + [''] * 3)[:3]

        # リプライデータの準備
        reply_data = []
        for reply in (received_replies + [{'text': '', 'created_at': ''}] * 3)[:3]:
            reply_data.extend([reply['text'], reply['created_at']])

        row = [
            profile['name'],
            profile['screen_name'],
            profile['followers_count'],
            profile['friends_count'],
            profile['statuses_count'],
            user_data['reply_count'],
            profile['description'],
            profile.get('location', ''),
            profile.get('url', ''),
            *tweets,           # 最新の3件のツイート
            *reply_data       # 相手からのリプライ3件分（テキストと日時）
        ]
        sheet.append(row)

    # 列幅の自動調整
    for column in sheet.columns:
        max_length = 0
        column_letter = get_column_letter(column[0].column)
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 50)  # 最大幅を50文字に制限
        sheet.column_dimensions[column_letter].width = adjusted_width

    # ファイルを保存
    filename = f"リプライ分析_{target_user}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    workbook.save(filename)
    print(f"Excelファイルを保存しました: {filename}")

# Google Colab用の実行コード
output.enable_custom_widget_manager()  # Google Colabでカスタムウィジェットを有効にする

# 分析の実行
await run_analysis()

認証に成功しました！
分析対象のユーザー名を入力してください（@を除く）: sora19ai
最小リプライ数を入力してください（例: 3）: 2
分析するツイート数を入力してください（例: 200）: 200

sora19aiのリプライを分析します...
- 分析対象ツイート数: 200
- 最小リプライ数: 2
sora19aiのツイートを分析中...
ツイートを取得中...
ユーザー 1108k4xi の情報を取得しました
20件のツイートを分析済み
ユーザー AquaCleanWater の情報を取得しました
ユーザー kawa11japan の情報を取得しました
40件のツイートを分析済み
60件のツイートを分析済み
80件のツイートを分析済み
ユーザー solt152152 の情報を取得しました
100件のツイートを分析済み
120件のツイートを分析済み
140件のツイートを分析済み
160件のツイートを分析済み
180件のツイートを分析済み
200件のツイートを分析済み

分析完了: 209件のツイートを処理
リプライ先ユーザー数: 4人

リプライの多いユーザーの情報を収集中...
@1108k4xiの情報を取得しました（リプライ数: 2）
分析結果を保存しました: /content/drive/MyDrive/Twitter_Analysis/profile_results/analysis_sora19ai_20241229_220205.json

Excelファイルを保存しました: /content/drive/MyDrive/Twitter_Analysis/profile_results/リプライ分析_sora19ai_20241229_220205.xlsx

分析が完了しました！


In [ ]:
# 非同期で分析を実行する関数
async def run_analysis():
        # TwitterProfileAnalyzerクラスのインスタンスを作成
        analyzer = TwitterProfileAnalyzer()

        # セットアップを実行し、認証が成功したか確認
        if not await analyzer.setup():
            return  # 認証に失敗した場合は関数を終了

        # ユーザー入力を対話的に取得
        target_user = input("分析対象のユーザー名を入力してください（@を除く）: ")  # 分析対象のTwitterユーザー名を入力
        min_replies = int(input("最小リプライ数を入力してください（例: 3）: "))  # リプライの最小数を入力
        tweets_to_analyze = int(input("分析するツイート数を入力してください（例: 200）: "))  # 分析するツイートの数を入力

        # 分析の開始をユーザーに通知
        print(f"\n{target_user}のリプライを分析します...")
        print(f"- 分析対象ツイート数: {tweets_to_analyze}")
        print(f"- 最小リプライ数: {min_replies}")

        # 指定したユーザーのリプライを分析
        reply_counter, reply_users = await analyzer.analyze_user_replies(target_user, tweets_to_analyze)

        # リプライが見つからなかった場合の処理
        if not reply_counter:
            print("\nリプライが見つかりませんでした。")
            return  # 関数を終了

        # 頻繁にリプライしているユーザーの情報を収集するためのリストを初期化
        frequent_repliers_data = []
        print("\nリプライの多いユーザーの情報を収集中...")

        # リプライカウントが多い順にユーザーを処理
        for screen_name, reply_count in reply_counter.most_common():
            # 最小リプライ数以上かつユーザー情報が存在する場合に処理
            if reply_count >= min_replies and screen_name in reply_users:
                try:
                    # リプライ先のユーザーオブジェクトを取得
                    user = reply_users[screen_name]
                    # ユーザーのプロフィール情報を取得
                    profile_data = await analyzer.get_user_profile(user)
                    # ユーザーの最新3件のツイートを取得
                    tweets = await analyzer.get_user_tweets(user, count=3)

                    # 相手からのリプライを取得（Twikitを使用）
                    received_replies = []
                    try:
                        # 検索クエリを構成
                        query = f"from:{screen_name} to:{target_user}"
                        # Twikitのsearch_tweet関数を使用
                        search_results = await analyzer.client.search_tweet(query, "Latest", count=20)

                        for tweet in search_results:
                            # ツイートがリプライであることを確認
                            tweet_data = tweet.__dict__.get('data', {})
                            referenced_tweets = tweet_data.get('referenced_tweets', [])

                            is_reply = any(ref.get('type') == 'replied_to' for ref in referenced_tweets)
                            if is_reply:
                                received_replies.append({
                                    'text': tweet.text,
                                    'created_at': tweet.created_at,
                                    'id': tweet.id
                                })

                            if len(received_replies) >= 3:  # 最大3件まで取得
                                break

                    except Exception as e:
                        print(f"リプライの取得中にエラーが発生: {e}")
                        received_replies = []

                    # プロフィール情報が取得できた場合にデータを構築
                    if profile_data:
                        user_data = {
                            'profile': profile_data,            # ユーザープロフィール情報
                            'reply_count': reply_count,         # リプライ数
                            'recent_tweets': tweets,            # 最近のツイート
                            'received_replies': received_replies  # 相手からのリプライ（新機能）
                        }
                        # 頻繁なリプライユーザーのリストに追加
                        frequent_repliers_data.append(user_data)
                        # 情報取得完了を通知
                        print(f"@{screen_name}の情報を取得しました（リプライ数: {reply_count}）")
                except Exception as e:
                    # ユーザー情報の取得に失敗した場合のエラーメッセージ
                    print(f"@{screen_name}の情報取得に失敗: {e}")
                    continue  # 次のユーザーに進む

        # 収集したデータが存在する場合に結果を保存
        if frequent_repliers_data:
            # JSONファイルとして保存
            analyzer.save_results(frequent_repliers_data, target_user)
            # Excelファイルとして保存
            analyzer.save_to_excel(frequent_repliers_data, target_user)
            # 分析完了を通知
            print("\n分析が完了しました！")
        else:
            # 分析対象のユーザーが見つからなかった場合のメッセージ
            print("\n分析対象となるユーザーが見つかりませんでした。")

# Excelファイルに保存する関数
def save_to_excel(self, data, target_user):
    """分析結果をExcelファイルに保存する"""
    workbook = Workbook()
    sheet = workbook.active
    sheet.title = "分析結果"

    # ヘッダー行を作成
    headers = [
        'ユーザー名', 'スクリーンネーム', 'フォロワー数', 'フォロー数',
        'ツイート数', 'リプライ数', '説明', '場所', 'URL',
        '最新ツイート1', '最新ツイート2', '最新ツイート3',
        '相手からのリプライ1', '相手からのリプライ日時1',
        '相手からのリプライ2', '相手からのリプライ日時2',
        '相手からのリプライ3', '相手からのリプライ日時3'
    ]
    sheet.append(headers)

    # データを行として追加
    for user_data in data:
        profile = user_data['profile']
        recent_tweets = user_data['recent_tweets']
        received_replies = user_data.get('received_replies', [])

        # ツイートとリプライの配列を3件に調整
        tweets = (recent_tweets + [''] * 3)[:3]

        # リプライデータの準備
        reply_data = []
        for reply in (received_replies + [{'text': '', 'created_at': ''}] * 3)[:3]:
            reply_data.extend([reply['text'], reply['created_at']])

        row = [
            profile['name'],
            profile['screen_name'],
            profile['followers_count'],
            profile['friends_count'],
            profile['statuses_count'],
            user_data['reply_count'],
            profile['description'],
            profile.get('location', ''),
            profile.get('url', ''),
            *tweets,           # 最新の3件のツイート
            *reply_data       # 相手からのリプライ3件分（テキストと日時）
        ]
        sheet.append(row)

    # 列幅の自動調整
    for column in sheet.columns:
        max_length = 0
        column_letter = get_column_letter(column[0].column)
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 50)  # 最大幅を50文字に制限
        sheet.column_dimensions[column_letter].width = adjusted_width

    # ファイルを保存
    filename = f"リプライ分析_{target_user}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    workbook.save(filename)
    print(f"Excelファイルを保存しました: {filename}")

# Google Colab用の実行コード
output.enable_custom_widget_manager()  # Google Colabでカスタムウィジェットを有効にする

# 分析の実行
await run_analysis()

認証に成功しました！
分析対象のユーザー名を入力してください（@を除く）: sora19ai
最小リプライ数を入力してください（例: 3）: 2
分析するツイート数を入力してください（例: 200）: 200

sora19aiのリプライを分析します...
- 分析対象ツイート数: 200
- 最小リプライ数: 2
sora19aiのツイートを分析中...
ツイートを取得中...
ユーザー 1108k4xi の情報を取得しました
20件のツイートを分析済み
ユーザー AquaCleanWater の情報を取得しました
ユーザー kawa11japan の情報を取得しました
40件のツイートを分析済み
60件のツイートを分析済み
80件のツイートを分析済み
ユーザー solt152152 の情報を取得しました
100件のツイートを分析済み
120件のツイートを分析済み
140件のツイートを分析済み
160件のツイートを分析済み
180件のツイートを分析済み
200件のツイートを分析済み

分析完了: 209件のツイートを処理
リプライ先ユーザー数: 4人

リプライの多いユーザーの情報を収集中...
@1108k4xiの情報を取得しました（リプライ数: 2）
分析結果を保存しました: /content/drive/MyDrive/Twitter_Analysis/profile_results/analysis_sora19ai_20241229_220527.json

Excelファイルを保存しました: /content/drive/MyDrive/Twitter_Analysis/profile_results/リプライ分析_sora19ai_20241229_220527.xlsx

分析が完了しました！


相手からのリプライ数　３回

In [ ]:
from collections import defaultdict, Counter
import os
from datetime import datetime
from openpyxl import Workbook
from openpyxl.utils import get_column_letter

# 非同期で分析を実行する関数
async def run_analysis():
        # TwitterProfileAnalyzerクラスのインスタンスを作成
        analyzer = TwitterProfileAnalyzer()

        # セットアップを実行し、認証が成功したか確認
        if not await analyzer.setup():
            return  # 認証に失敗した場合は関数を終了

        # ユーザー入力を対話的に取得
        target_user = input("分析対象のユーザー名を入力してください（@を除く）: ")
        min_replies = int(input("最小リプライ数を入力してください（例: 3）: "))
        tweets_to_analyze = int(input("分析するツイート数を入力してください（例: 200）: "))

        print(f"\n{target_user}へのリプライを分析します...")
        print(f"- 分析対象ツイート数: {tweets_to_analyze}")
        print(f"- 最小リプライ数: {min_replies}")

        # 相手からのリプライを分析
        reply_counter = Counter()  # リプライ数をカウントするカウンター
        reply_users = {}  # リプライしてきたユーザーの情報を保持
        reply_contents = defaultdict(list)  # ユーザーごとのリプライ内容を保持

        try:
            # 過去のツイートを検索して相手からのリプライを取得
            query = f"to:{target_user}"
            search_results = await analyzer.client.search_tweet(query, "Latest", count=100)

            for tweet in search_results:
                try:
                    author = tweet.user.screen_name
                    # リプライ数をカウント
                    reply_counter[author] += 1
                    # ユーザー情報を保存
                    if author not in reply_users:
                        reply_users[author] = tweet.user
                    # リプライ内容を保存（最大3件まで）
                    if len(reply_contents[author]) < 3:
                        reply_contents[author].append({
                            'text': tweet.text,
                            'created_at': str(tweet.created_at),
                            'id': tweet.id
                        })
                except AttributeError:
                    continue

        except Exception as e:
            print(f"リプライの検索中にエラー: {e}")
            return

        # リプライが見つからなかった場合の処理
        if not reply_counter:
            print("\nリプライが見つかりませんでした。")
            return

        # 頻繁にリプライしているユーザーの情報を収集
        frequent_repliers_data = []
        print("\nリプライの多いユーザーの情報を収集中...")

        # リプライカウントが多い順にユーザーを処理
        for screen_name, reply_count in reply_counter.most_common():
            if reply_count >= min_replies:
                try:
                    user = reply_users[screen_name]
                    # ユーザーのプロフィール情報を取得
                    profile_data = await analyzer.get_user_profile(user)
                    # 最新のツイートを取得
                    tweets = await analyzer.get_user_tweets(user, count=3)

                    if profile_data:
                        user_data = {
                            'profile': profile_data,
                            'reply_count': reply_count,  # 相手からのリプライ数
                            'recent_tweets': tweets,
                            'received_replies': reply_contents[screen_name]  # 保存したリプライ内容
                        }
                        frequent_repliers_data.append(user_data)
                        print(f"@{screen_name}の情報を取得しました（リプライ数: {reply_count}）")

                except Exception as e:
                    print(f"@{screen_name}の情報取得に失敗: {e}")
                    continue

        # 収集したデータが存在する場合に結果を保存
        if frequent_repliers_data:
            analyzer.save_results(frequent_repliers_data, target_user)
            analyzer.save_to_excel(frequent_repliers_data, target_user)
            print("\n分析が完了しました！")
        else:
            print("\n分析対象となるユーザーが見つかりませんでした。")

def save_to_excel(self, data, target_user):
    """分析結果をExcelファイルに保存する"""
    workbook = Workbook()
    sheet = workbook.active
    sheet.title = "分析結果"

    headers = [
        'ユーザー名', 'スクリーンネーム', 'フォロワー数', 'フォロー数',
        'ツイート数', '相手からのリプライ数', '説明', '場所', 'URL',
        '最新ツイート1', '最新ツイート2', '最新ツイート3',
        '相手からのリプライ1', '相手からのリプライ日時1',
        '相手からのリプライ2', '相手からのリプライ日時2',
        '相手からのリプライ3', '相手からのリプライ日時3'
    ]
    sheet.append(headers)

    for user_data in data:
        profile = user_data['profile']
        recent_tweets = [tweet.text if hasattr(tweet, 'text') else str(tweet) for tweet in user_data['recent_tweets']]
        received_replies = user_data.get('received_replies', [])

        tweets = (recent_tweets + [''] * 3)[:3]

        reply_data = []
        for reply in (received_replies + [{'text': '', 'created_at': ''}] * 3)[:3]:
            reply_data.extend([reply['text'], reply['created_at']])

        row = [
            profile['name'],
            profile['screen_name'],
            profile['followers_count'],
            profile['friends_count'],
            profile['statuses_count'],
            user_data['reply_count'],  # 相手からのリプライ数
            profile['description'],
            profile.get('location', ''),
            profile.get('url', ''),
            *tweets,
            *reply_data
        ]
        sheet.append(row)

    for column in sheet.columns:
        max_length = 0
        column_letter = get_column_letter(column[0].column)
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 50)
        sheet.column_dimensions[column_letter].width = adjusted_width

    filename = f"リプライ分析_{target_user}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    save_path = os.path.join(os.getcwd(), filename)
    workbook.save(save_path)
    print(f"Excelファイルを保存しました: {save_path}")

await run_analysis()

認証に成功しました！
分析対象のユーザー名を入力してください（@を除く）: sora19ai
最小リプライ数を入力してください（例: 3）: 2
分析するツイート数を入力してください（例: 200）: 200

sora19aiへのリプライを分析します...
- 分析対象ツイート数: 200
- 最小リプライ数: 2

リプライの多いユーザーの情報を収集中...
@sora19aiの情報を取得しました（リプライ数: 9）
@seikimitsumoriの情報を取得しました（リプライ数: 2）
分析結果を保存しました: /content/drive/MyDrive/Twitter_Analysis/profile_results/analysis_sora19ai_20241229_221050.json

Excelファイルを保存しました: /content/drive/MyDrive/Twitter_Analysis/profile_results/リプライ分析_sora19ai_20241229_221050.xlsx

分析が完了しました！


テスト

# キーワード検索

キーワード検索　初期設定のセル

In [ ]:
# 必要なライブラリのインストール
!pip install twikit pandas openpyxl

# 必要なインポート
from twikit import Client
import json
import os
from datetime import datetime
import asyncio
import pandas as pd
from google.colab import drive
from google.colab import files

# Google Driveのマウント
drive.mount('/content/drive')

# ディレクトリ設定
WORK_DIR = '/content/drive/MyDrive/Twitter_Analysis'
os.makedirs(WORK_DIR, exist_ok=True)

COOKIE_DIR = f"{WORK_DIR}/twitter_json"
os.makedirs(COOKIE_DIR, exist_ok=True)

RESULTS_DIR = f"{WORK_DIR}/keyword_search_results"
os.makedirs(RESULTS_DIR, exist_ok=True)

print("ディレクトリ設定完了！")
print(f"作業ディレクトリ: {WORK_DIR}")
print(f"クッキー保存先: {COOKIE_DIR}")
print(f"結果保存先: {RESULTS_DIR}")

# クッキーファイルのアップロード
def upload_cookie_file():
    print("cookie_edit.jsonファイルをアップロードしてください")
    uploaded = files.upload()

    if 'cookie_edit.json' in uploaded:
        with open(f"{COOKIE_DIR}/cookie_edit.json", 'wb') as f:
            f.write(uploaded['cookie_edit.json'])
        print("クッキーファイルを保存しました")
        return True
    else:
        print("cookie_edit.jsonファイルがアップロードされませんでした")
        return False

upload_cookie_file()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ディレクトリ設定完了！
作業ディレクトリ: /content/drive/MyDrive/Twitter_Analysis
クッキー保存先: /content/drive/MyDrive/Twitter_Analysis/twitter_json
結果保存先: /content/drive/MyDrive/Twitter_Analysis/keyword_search_results
cookie_edit.jsonファイルをアップロードしてください


Saving cookie_edit.json to cookie_edit (1).json
cookie_edit.jsonファイルがアップロードされませんでした


False

キーワード検索　クラス定義セル

In [ ]:
class TwitterKeywordAnalyzer:
    def __init__(self):
        self.client = Client(language='en-US')
        self.cookie_path = f"{COOKIE_DIR}/cookie_edit.json"
        self.results_dir = RESULTS_DIR

    async def setup(self):
        try:
            with open(self.cookie_path, 'r', encoding='utf-8') as file:
                cookies = json.load(file)
            self.client.set_cookies(cookies)
            print("認証に成功しました！")
            return True
        except Exception as e:
            print(f"認証エラー: {e}")
            return False

    async def search_with_keyword(self, keyword, count=20, sort_by='latest'):
        """キーワードを含むツイートを検索し、関連情報を取得する"""
        try:
            search_results = []
            print(f"'{keyword}' に関連する情報を検索中...(並び順: {sort_by})")

            product_type = 'Latest' if sort_by == 'latest' else 'Top'
            results = await self.client.search_tweet(
                query=keyword,
                product=product_type,
                count=min(count, 20)
            )

            for tweet in results:
                user_data = {
                    'user_id': tweet.user.id,
                    'name': tweet.user.name,
                    'screen_name': tweet.user.screen_name,
                    'profile_description': tweet.user.description,
                    'profile_url': f"https://twitter.com/{tweet.user.screen_name}",
                    'followers_count': tweet.user.followers_count,
                    'following_count': tweet.user.following_count,
                    'profile_image_url': tweet.user.profile_image_url,
                    'location': tweet.user.location
                }

                tweet_data = {
                    'tweet_id': tweet.id,
                    'tweet_url': f"https://twitter.com/{tweet.user.screen_name}/status/{tweet.id}",
                    'text': tweet.text,
                    'created_at': tweet.created_at,
                    'retweet_count': tweet.retweet_count,
                    'like_count': tweet.favorite_count,
                    'reply_count': tweet.reply_count if hasattr(tweet, 'reply_count') else 0,
                    'is_retweet': bool(tweet.retweeted_tweet),
                    'is_quote': tweet.is_quote_status,
                    'language': tweet.lang
                }

                keyword_locations = []
                if keyword.lower() in tweet.text.lower():
                    keyword_locations.append('tweet_text')
                if keyword.lower() in tweet.user.description.lower():
                    keyword_locations.append('profile_description')
                if keyword.lower() in tweet.user.name.lower():
                    keyword_locations.append('user_name')
                if keyword.lower() in tweet.user.screen_name.lower():
                    keyword_locations.append('screen_name')

                search_results.append({
                    'user': user_data,
                    'tweet': tweet_data,
                    'keyword_locations': keyword_locations
                })

            if sort_by == 'likes':
                search_results.sort(key=lambda x: x['tweet']['like_count'], reverse=True)

            return search_results

        except Exception as e:
            print(f"検索エラー: {e}")
            return []

    # save_results と save_to_excel メソッドはそのまま
    def save_results(self, results, filename_prefix):
        """検索結果をJSONファイルとして保存する"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{self.results_dir}/{filename_prefix}_{timestamp}.json"

        try:
            with open(filename, 'w', encoding='utf-8') as file:
                json.dump(results, file, ensure_ascii=False, indent=2, default=str)
            print(f"\n結果を保存しました: {filename}")
        except Exception as e:
            print(f"結果の保存中にエラーが発生しました: {e}")

    def save_to_excel(self, results, keyword, sort_by):
        """検索結果をExcelファイルとして保存"""
        try:
            # 結果をDataFrame用に整形
            rows = []
            for result in results:
                user = result['user']
                tweet = result['tweet']
                locations = result['keyword_locations']

                row = {
                    '検索日時': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    '検索キーワード': keyword,
                    '並び順': sort_by,
                    '投稿日時': tweet['created_at'],
                    'アカウント名': user['name'],
                    'ユーザーID': f"@{user['screen_name']}",
                    'プロフィール文': user['profile_description'],
                    'フォロワー数': user['followers_count'],
                    'フォロー数': user['following_count'],
                    'ツイート本文': tweet['text'],
                    'いいね数': tweet['like_count'],
                    'リツイート数': tweet['retweet_count'],
                    'リプライ数': tweet['reply_count'],
                    'ツイートURL': tweet['tweet_url'],
                    'アカウントURL': user['profile_url'],
                    'キーワード出現場所': ', '.join(locations),
                    '場所': user['location'] if user['location'] else '',
                    '言語': tweet['language']
                }
                rows.append(row)

            # DataFrameを作成
            df = pd.DataFrame(rows)

            # Excelファイル名を生成
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            excel_file = f"{self.results_dir}/Twitter検索結果_{keyword}_{sort_by}_{timestamp}.xlsx"

            # Excelファイルとして保存
            with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
                df.to_excel(writer, sheet_name='検索結果', index=False)

                # 列幅の自動調整
                worksheet = writer.sheets['検索結果']
                for idx, col in enumerate(df.columns):
                    max_length = max(
                        df[col].astype(str).apply(len).max(),
                        len(str(col))
                    )
                    # 最大幅を50文字に制限
                    adjusted_width = min(max_length + 2, 50)
                    worksheet.column_dimensions[chr(65 + idx)].width = adjusted_width

            print(f"\nExcelファイルを保存しました: {excel_file}")
            return excel_file

        except Exception as e:
            print(f"Excelファイルの保存中にエラーが発生しました: {e}")
            return None

キーワード検索　実行セル

In [ ]:
async def run_search():
    analyzer = TwitterKeywordAnalyzer()

    if not await analyzer.setup():
        return

    # 検索設定を対話的に取得
    keyword = input("検索キーワードを入力してください: ")
    count = int(input("取得する結果の数を入力してください（最大20）: "))

    print("\n検索オプション:")
    print("1: 新しい順")
    print("2: 人気順")
    print("3: いいね数順")

    try:
        option = int(input("検索オプションを選択してください (1-3): "))
        sort_by = {
            1: 'latest',
            2: 'top',
            3: 'likes'
        }.get(option, 'latest')
    except ValueError:
        print("無効な入力です。デフォルトの'新しい順'で検索します。")
        sort_by = 'latest'

    results = await analyzer.search_with_keyword(keyword, count, sort_by)

    if not results:
        print("検索結果が見つかりませんでした。")
        return

    sort_type = {
        'latest': '新しい順',
        'top': '人気順',
        'likes': 'いいね数順'
    }[sort_by]

    print(f"\n検索結果 ({len(results)} 件) - {sort_type}:")
    for result in results:
        print("\n" + "="*50)
        user = result['user']
        tweet = result['tweet']

        print(f"投稿日時: {tweet['created_at']}")
        print(f"いいね数: {tweet['like_count']}")
        print(f"\nユーザー情報: {user['name']} (@{user['screen_name']})")
        print(f"ツイート: {tweet['text']}")
        print(f"URL: {tweet['tweet_url']}")

    if results:
        analyzer.save_results(results, f"{keyword}_{sort_by}")
        analyzer.save_to_excel(results, keyword, sort_type)

# Google Colab用の実行
from google.colab import output
output.enable_custom_widget_manager()

# 分析実行
await run_search()

認証に成功しました！
検索キーワードを入力してください: dify
取得する結果の数を入力してください（最大20）: 20

検索オプション:
1: 新しい順
2: 人気順
3: いいね数順
検索オプションを選択してください (1-3): 2
'dify' に関連する情報を検索中...(並び順: top)

検索結果 (19 件) - 人気順:

投稿日時: Wed Dec 11 03:06:26 +0000 2024
いいね数: 6

ユーザー情報: katoshin (@katoshin_ii)
ツイート: 要件定義を対話型で実現するDifyのチャットフローがなかなか大作になってしまった。 https://t.co/JKJNpFFn4A
URL: https://twitter.com/katoshin_ii/status/1866680917722632626

投稿日時: Sat Dec 07 07:53:30 +0000 2024
いいね数: 171

ユーザー情報: Tom | ドバイで生成AIやってる人 (@0x__tom)
ツイート: Difyを使えば、「食事や体重の記録」から「記録に基づいたアドバイスの生成」までを一瞬で自動化できる。写真をアップするだけで、カロリー計算や体重をスプシに記録し、AIがデータを分析してパーソナライズされたアドバイスを提案。面倒な記録の手間を省き、健康管理をサポートしてくれます。 https://t.co/3YkdhDSbAb
URL: https://twitter.com/0x__tom/status/1865303606683164768

投稿日時: Wed Dec 11 16:35:55 +0000 2024
いいね数: 5

ユーザー情報: 森本洋平@すもちゃん| 動画×AI・Dify開発 (@sumo_wonderful)
ツイート: Soraの記念すべき初体験AI生成は「sumo wrestler」と入力してストーリーもAIに自動作成してもらった。

いくらすごいAI動画生成ツールでも"相撲"の学習量と知名度が少なすぎるんだろうね。
AIツールを最初に試す時、相撲関連のプロンプト当てると大体そのAIツールの限界が見える。 https://t.co/4WiuQOHgNC
URL: https://twitt